In [ ]:
import massbalancemachine as mbm
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('./example_data/iceland/files/iceland_monthly_dataset.csv')
df.dropna(axis=0, how='any', inplace=True)

In [ ]:
dataloader = mbm.DataLoader(data=df)

In [ ]:
train_itr, test_itr = dataloader.set_train_test_split()

In [ ]:
splits = dataloader.get_cv_split()

In [ ]:
parameters = {
    'max_depth': [3, 4, 5,],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'gamma':[0, 1]
}

clf = GridSearchCV(
    estimator=...,
    param_grid=parameters,
    cv=splits,
    verbose=10, 
    n_jobs=-1, 
    refit=True, 
    return_train_score=True,
    error_score='raise'
)

# Fit using features
clf.fit(..., ...)